In [ ]:
test = '7. Borussia M.Gladbach'
test.split(' ', 1)[1]

In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

save_dir = "./Leagues"
targets = ['England-Premier-League', 'Spain-LaLiga', 'Italy-Serie-A', 'Germany-Bundesliga', 'France-Ligue-1']

contents = {'England-Premier-League' : 'standings-20934-content', 
            'Spain-LaLiga' : 'standings-21073-content', 
            'Italy-Serie-A' : 'standings-21087-content', 
            'Germany-Bundesliga' : 'standings-21026-content', 
            'France-Ligue-1' : 'standings-21037-content'}

url = 'https://www.whoscored.com/'
browser = webdriver.Chrome()
browser.get(url)

browser.find_element(By.ID, 'Top-Tournaments-btn').click()
tournaments = browser.find_elements(By.CLASS_NAME, 'TournamentNavButton-module_clickableArea__ZFnBl')
# tournaments = [t.get_attribute('href') for t in tournaments] 
# print(tournaments)  

counter = 1
for tournament in tournaments:
    if counter > len(targets):
        break

    link = tournament.get_attribute('href')
    league_name = link.split('/')[-1]

    if league_name in targets:
        if not os.path.exists(f"{save_dir}/{league_name}"): 
            os.makedirs(f"{save_dir}/{league_name}")

        browser2 = webdriver.Chrome()
        wait = WebDriverWait(browser2, 10)
        browser2.get(link)

        select = Select(browser2.find_element(By.ID,'seasons'))
        select.select_by_visible_text("2022/2023")

        if league_name == 'Italy-Serie-A':
            select = Select(browser2.find_element(By.ID,'stages'))
            select.select_by_visible_text("Serie A")

        # time.sleep(1.5)
        table = browser2.find_element(By.ID, contents[league_name])
        rows = table.find_elements(By.TAG_NAME, 'tr')

        dataset = []
        for row in rows:
            team = row.find_element(By.CLASS_NAME, "team-link")
            team_name = team.text
            print(team_name)
            team_url = team.get_attribute('href')

            num_matches = row.find_element(By.CLASS_NAME, "p").text
            num_win = row.find_element(By.CLASS_NAME, 'w').text
            num_draw = row.find_element(By.CLASS_NAME, 'd').text
            num_lose = row.find_element(By.CLASS_NAME, 'l').text

            ## gf : 득점, ga : 실점, gd : 득실차, pts : 승점
            goal_for = row.find_element(By.CLASS_NAME, 'gf').text
            goal_against = row.find_element(By.CLASS_NAME, 'ga').text
            goal_difference = row.find_element(By.CLASS_NAME, 'gd').text
            points = row.find_element(By.CLASS_NAME, 'pts').text

            dataset.append([team_name, num_matches, num_win, num_draw, num_lose, goal_for, goal_against, goal_difference, points])
        teams_df = pd.DataFrame(dataset, columns=['team_name', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'])

        sub_nav = browser2.find_element(By.ID, 'sub-navigation')
        nav_items = sub_nav.find_elements(By.TAG_NAME, 'li')

        ## Team Stat
        team_stat_btn = nav_items[2].click()
        time.sleep(2)

        team_stat_table = browser2.find_element(By.ID, 'top-team-stats-summary-content')
        team_stat_rows = team_stat_table.find_elements(By.TAG_NAME, 'tr')

        total_stats = []
        for idx, row in enumerate(team_stat_rows):
            team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
            goal = row.find_element(By.CLASS_NAME, 'goal').text
            shots_per_game = row.find_element(By.CLASS_NAME, 'shotsPerGame').text
            yellow_cards = row.find_element(By.CLASS_NAME, 'yellow-card-box').text
            red_cards = row.find_element(By.CLASS_NAME, 'red-card-box').text
            possession = row.find_element(By.CLASS_NAME, 'possession').text
            pass_success = row.find_element(By.CLASS_NAME, 'passSuccess').text
            aerial_won_per_game = row.find_element(By.CLASS_NAME, 'aerialWonPerGame').text
            rating = row.find_element(By.CLASS_NAME, 'rating').text

            total_stats.append([team_name, goal, shots_per_game, yellow_cards, red_cards, possession, pass_success, aerial_won_per_game, rating])

        total_stats_df = pd.DataFrame(total_stats, columns=['team_name', 'Goals', 'Shots pg', 'Yellow', 'Red', 'Poss%', 'Pass%', 'A_Won', 'Rating'])

        final_df = pd.merge(teams_df, total_stats_df, on='team_name', how='left')
        browser2.close()
        counter += 1

        final_df.to_csv(f'{save_dir}/{league_name}/{league_name}_22_23.csv')

browser.close()

In [1]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


save_dir = "./Leagues"

urls = [
    'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/TeamStatistics/Germany-Bundesliga-2022-2023',
    'https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/TeamStatistics/Spain-LaLiga-2022-2023',
    'https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/TeamStatistics/France-Ligue-1-2022-2023',
    'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/TeamStatistics/England-Premier-League-2022-2023',
    'https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/TeamStatistics/Italy-Serie-A-2022-2023',
    ]


teams_df_list = [
    f"./{save_dir}/Germany-Bundesliga/Germany-Bundesliga_22_23.csv",
    f"./{save_dir}/Spain-LaLiga/Spain-LaLiga_22_23.csv",
    f"./{save_dir}/France-Ligue-1/France-Ligue-1_22_23.csv",
    f"./{save_dir}/England-Premier-League/England-Premier-League_22_23.csv",
    f"./{save_dir}/Italy-Serie-A/Italy-Serie-A_22_23.csv"
    ]


for idx, url in enumerate(urls):
    league_name = teams_df_list[idx].split('/')[-2]
    print(url, f"{save_dir}/{league_name}/{league_name}")

    team_df = pd.read_csv(teams_df_list[idx])
    final_df = team_df

    browser = webdriver.Chrome()
    browser.get(url)
    wait = WebDriverWait(browser, 10)

    options_container = wait.until(EC.presence_of_element_located((By.ID, 'stage-team-stats-options')))
    list_items = options_container.find_elements(By.TAG_NAME, 'li')

    defensive_stat = []
    offensive_stat = []
    xg_stat = []
    detailed_stat = []
    for list_item in list_items[1:]:
        item_name = list_item.text
        # browser.execute_script("arguments[0].click();", list_item)
        list_item.click()
        time.sleep(3)

        container = browser.find_element(By.ID, f'stage-team-stats-{item_name.lower()}')
        print(f'stage-team-stats-{item_name.lower()}')

        table = container.find_element(By.CLASS_NAME, 'with-centered-columns')
        # headers = table.find_elements(By.TAG_NAME, 'th')
        # columns = [header.text for header in headers]
        time.sleep(3)

        body = table.find_element(By.ID, 'top-team-stats-summary-content')
        rows = body.find_elements(By.TAG_NAME, 'tr')

        if item_name == "Defensive":
            data = []
            columns = ['team_name', 'Shoted pg', 'Tackles pg', 'Intercept pg', 'Fouls pg', 'Offsides pg']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                shots_pg = row.find_element(By.CLASS_NAME, 'shotsConcededPerGame').text
                tackles_pg = row.find_element(By.CLASS_NAME, 'tacklePerGame').text
                interceptions_pg = row.find_element(By.CLASS_NAME, 'interceptionPerGame').text
                fouls_pg = row.find_element(By.CLASS_NAME, 'foulsPerGame').text
                offsides_pg = row.find_element(By.CLASS_NAME, 'offsideGivenPerGame').text

                data.append([team_name, shots_pg, tackles_pg, interceptions_pg, fouls_pg, offsides_pg])
            defensive_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, defensive_stat, on='team_name', how='left')

        if item_name == 'Offensive':
            data = []
            columns = ['team_name', 'Shots OT pg', 'Dribbles pg', 'Fouled pg']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                shot_on_target_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text
                dribble_won_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text
                foul_given_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text

                data.append([team_name, shot_on_target_per_game, dribble_won_per_game, foul_given_per_game])
            offensive_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, offensive_stat, on='team_name', how='left')
        
        if item_name == 'xG':
            data = []
            columns = ['team_name', 'xG', 'Goals', 'xGDiff', 'Shots', 'xG/shots']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                xg = row.find_element(By.CLASS_NAME, 'xG').text
                goal_exc_own = row.find_element(By.CLASS_NAME, 'goalExcOwn').text
                xgdiff = row.find_element(By.CLASS_NAME, 'xGDiff').text
                total_shots = row.find_element(By.CLASS_NAME, 'totalShots').text
                xg_per_shot = row.find_element(By.CLASS_NAME, 'xGPerShot').text

                data.append([team_name, xg, goal_exc_own, xgdiff, total_shots, xg_per_shot])
            xg_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, xg_stat, on='team_name', how='left')

        if item_name == "Detailed":
            data = []
            columns = ['team_name', 'shotsTotal', 'shot0boxTotal', 'shotSixYardBox', 'shotPentaltyArea']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                shots_total = row.find_element(By.CLASS_NAME, 'shotsTotal').text
                shots_0box_total = row.find_element(By.CLASS_NAME, 'shotOboxTotal').text
                shots_six_yard_box = row.find_element(By.CLASS_NAME, 'shotSixYardBox').text
                shots_penalty_area = row.find_element(By.CLASS_NAME, 'shotPenaltyArea').text

                data.append([team_name, shots_total, shots_0box_total, shots_six_yard_box, shots_penalty_area])
            detailed_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, detailed_stat, on='team_name', how='left')

    final_df.to_csv(f"{save_dir}/{league_name}/{league_name}_22_23_complete.csv")
    browser.close()

https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/TeamStatistics/Germany-Bundesliga-2022-2023 ./Leagues/Germany-Bundesliga/Germany-Bundesliga
stage-team-stats-defensive
stage-team-stats-offensive
stage-team-stats-xg
stage-team-stats-detailed
https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/TeamStatistics/Spain-LaLiga-2022-2023 ./Leagues/Spain-LaLiga/Spain-LaLiga
stage-team-stats-defensive
stage-team-stats-offensive
stage-team-stats-xg
stage-team-stats-detailed
https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/TeamStatistics/France-Ligue-1-2022-2023 ./Leagues/France-Ligue-1/France-Ligue-1
stage-team-stats-defensive
stage-team-stats-offensive
stage-team-stats-xg
stage-team-stats-detailed
https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/TeamStatistics/England-Premier-League-2022-2023 ./Leagues/England-Premier-League/England-Premier-League
stage-team-stats-defensive
stage-team-s